# Import the libraries

In [ ]:
import numpy.random
import random
from itertools import product
from dagsim.base import Graph, Node

# Define the simulation using Python code

## Define the functions

In [ ]:

BASE_SEQ_FN = "rep.txt"
AIRR_SIZE = 1000


def _get_olga_seq(protocol):
    for line in open(BASE_SEQ_FN):
        seq = line.strip()
        if len(seq) < 10:
            continue
        if protocol == 1 and not seq.startswith("CAS"):
            continue
        yield seq


def assign_protocol(disease):
    return numpy.random.binomial(1, 0.1 + 0.8 * disease)


def create_airr(disease, age, protocol):
    airr = []
    left = AIRR_SIZE
    for seq in _get_olga_seq(protocol):
        if left == 0:
            break
        if disease == 1:
            seq = seq[0:5] + _get_signal() + seq[8:len(seq)]
        clono_size = _get_clono_size(age, left)
        left -= clono_size
        airr.append((seq, clono_size))
    assert left == 0
    return airr


def _get_clono_size(age, max_left):
    return int(min(numpy.random.lognormal((120 - age) / 20, 1.5), max_left))


def _get_signal():
    return random.choice(["CAT", "CAR", "CAS", "DOG"])


def encode_kmers(airr):
    alphabet = "ARNDCQEGHILKMFPOSUTWYVBZXJ"
    k = 3
    kmers = sorted(list([''.join(x) for x in product(*[alphabet] * k)]))
    counts = dict([(kmer, 0) for kmer in kmers])
    for seq, _ in airr:
        for i in range(len(seq) - k + 1):
            sub = seq[i:i + k]
            counts[sub] += 1
    occ_vector = [counts[kmer] for kmer in kmers]
    return occ_vector

## Define and draw the graph

In [ ]:
disease = Node(name="Disease", function=numpy.random.binomial, kwargs={"n": 1, "p": 0.5})
age = Node(name="Age", function=numpy.random.randint, kwargs={"low": 10, "high": 80})
protocol = Node(name="Protocol", function=assign_protocol, kwargs={"disease": disease})
airr = Node(name="AIRR", function=create_airr, kwargs={"disease": disease, "age": age, "protocol": protocol},
                  observed=True)
kmer_vec = Node(name="kmerVec", function=encode_kmers, kwargs={"airr": airr})

In [ ]:
graph = Graph(list_nodes=[disease, age, protocol, airr, kmer_vec])
graph.draw()

## Simulate repertoires

In [ ]:
data = graph.simulate(num_samples=50, csv_name="BioseqExample")

# Define the simulation using YAML

In [ ]:
from dagsim.utils.parser import DagSimSpec

data = DagSimSpec("BioseqExample.yaml").parse(verbose=False, draw=False)

# Train a Logistic Regression model

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np

X = np.array(data["kmerVec"])
print(X.shape)

y = np.array(data["Disease"])
print(y.shape)

n = 30

X_train = X[:n]
X_test =  X[n:]

y_train = y[:n]
y_test =  y[n:]


clf = LogisticRegression()
clf.fit(X_train, y_train)
print(clf.score(X_test,y_test))


## Get the kmers with the top-5 highest scores

In [ ]:
k = 3
alphabet = "ARNDCQEGHILKMFPOSUTWYVBZXJ"
kmers_list = sorted(list([''.join(x) for x in product(*[alphabet] * k)]))
print(kmers_list[:5])

In [ ]:
m = 5

clist = list(clf.coef_[0])
x = sorted(clist)[-m-2]

clist = [1 if c>x else 0 for c in clist]

klist = [kmers_list[i] for i in range(len(kmers_list)) if clist[i]==1]
print(klist)